<a href="https://colab.research.google.com/github/ericjin92/ej/blob/main/Steam_%E7%86%B1%E9%96%80%E9%81%8A%E6%88%B2%E6%8E%92%E8%A1%8C%E6%A6%9C%E8%88%87%E5%83%B9%E6%A0%BC%E8%B3%87%E8%A8%8A%E8%92%90%E9%9B%86%E5%AF%A6%E4%BD%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HW02 Web Crawler

## Steam 熱門遊戲排行榜與價格資訊蒐集實作

以 Steam 官方網站及第三方平台為主要資料來源，透過網頁爬蟲技術蒐集多項遊戲趨勢、價格與評價資訊，進一步分析全球steam遊戲市場熱度、價格變化與玩家偏好，為遊戲行銷與消費行為研究提供實證依據。

## 摘要

本報告旨在應用 Python 網頁爬蟲技術，自 Steam 官方網站及第三方平台（如 Steam Charts）擷取熱門遊戲的排行榜、價格、玩家數據與評價資訊。透過資料整理與視覺化分析，觀察全球與台灣地區 Steam 遊戲市場的熱門趨勢、定價策略與消費行為，期望為遊戲產業研究與行銷策略提供實證依據。

## 引言

隨著數位娛樂產業快速成長，Steam 已成為全球最大且影響力深遠的 PC 遊戲平台。玩家數據、銷售排行與價格波動等資訊，對於遊戲開發商與行銷單位來說皆具高度參考價值。

第三方平台如 Steam Charts 提供即時玩家統計，能夠反映出遊戲熱度與市場反應。透過自動化資料擷取技術，可高效率取得龐大資料，進一步進行視覺化與趨勢分析。

本次爬蟲的目標有兩部分：

* 擷取 Steam 上全球與台灣的暢銷排行榜遊戲資訊，分析地域間趨勢差異。
* 擷取 Steam Charts 上熱門遊戲的即時玩家數據，分析市場熱度與變化趨勢。

## 方法

- 目標網站描述
  - 目標網站：
    
   * Steam 暢銷遊戲排行榜：https://store.steampowered.com/charts/topselling/
   * Steam Charts 最多遊玩人數排行榜：https://steamcharts.com/top
  
  - 頁面結構：
   
   * Steam 暢銷排行榜：遊戲名稱、封面圖、目前價格、原價、評價比例、熱銷排名。
   * Steam Charts 玩家榜：遊戲名稱、目前玩家數、歷史高峰人數、總平均玩家數
   
- 工具與技術
  - 使用 Python 的 `requests` 庫發送 HTTP 請求。
  - 使用 `BeautifulSoup` 解析 HTML。
  - 使用 Google Chrome 開發者工具取得 AJAX 資料。
  - 使用 pandas 進行資料整理與儲存


## Steam 全球熱門暢銷遊戲 Top 100

先確認是否可以抓取資料， 200 為正確， 403 為禁止存取， 404 為網頁不存在， 503 為服務暫時無法使用。

In [63]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

popular_url = 'https://store.steampowered.com/search/?filter=globaltopsellers&cc=TW&ndl=1'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36 Edg/135.0.0.0',
       'Accept-Language': 'zh-TW,zh;q=0.9',
       'Cookie': 'steamCountry=TW; Steam_Language=tchinese'}
response = requests.get(popular_url,headers=headers)
response.status_code

200

藉由 Beautifulshop 抓取遊戲名稱、圖片網址、評價、價格、折扣幅度、遊戲連結，最後利用 pandas 製作表格存於 csv 檔中。

In [65]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# 用來儲存所有遊戲資料
data = []
# 初始排名
rank = 1

# Steam 一頁50筆，抓取前2頁共100筆
for page in range(0, 50, 25):
    popular_top100_url = f'https://store.steampowered.com/search/?filter=globaltopsellers&cc=TW&ndl=1&start={page}'
    r = requests.get(popular_top100_url, headers=headers)
    soup = BeautifulSoup(r.text, 'lxml')
    games = soup.find_all('a', class_='search_result_row')
    for game in games:
        # 頁面連結
        link = game['href']
        # 圖片
        img_tag = game.find('div', class_='col search_capsule').find('img')
        img = img_tag['src'] if img_tag else '無圖片資訊'
        # 名稱、價格、評價都在這個區塊
        name_price_div = game.find('div', class_='responsive_search_name_combined')
        # 名稱
        name_text = name_price_div.find('div', class_='col search_name ellipsis')
        name = name_text.find('span', class_='title').text.strip() if name_text else '無名稱資訊'
        # 價格
        price_div = name_price_div.find('div', class_='col search_price_discount_combined responsive_secondrow')
        if price_div.find('div', class_='discount_block search_discount_block'):
          discount = price_div.find('div', class_='discount_block search_discount_block')
          rate = discount.find('div', class_='discount_pct').text.strip() # 折扣幅度
          price_dis = discount.find('div', class_='discount_prices')
          price = price_dis.find('div', class_='discount_final_price').text.strip() # 折扣後價格
        elif price_div.find('div', class_='discount_block search_discount_block no_discount'):
           no_discount = price_div.find('div', class_='discount_block search_discount_block no_discount')
           rate = '無折扣'
           price_nodis = no_discount.find('div', class_='discount_prices')
           price = price_nodis.find('div', class_='discount_final_price').text.strip()
        elif price_div.find('div', class_='discount_final_price free'):
           rate = '無折扣'
           price = '免費'
        else:
           rate = '無折扣'
           price = '無價格資訊'
        # 評價
        review_div = name_price_div.find('div', class_='col search_reviewscore responsive_secondrow')
        if review_div and review_div.span and 'data-tooltip-html' in review_div.span.attrs:
            review_html = review_div.span['data-tooltip-html']
            review_rate = review_html.split('<br>')[0]
            review_tot = review_html.split('<br>')[-1]
        else:
          review_rate = '尚無評價'
          review_tot = '尚無評價'
        # 輸入資料到表格
        data.append({
            '排名': rank,
            '遊戲名稱': name,
            '圖片網址': img,
            '評價比例': review_rate,
            '總體評價': review_tot,
            '折扣幅度': rate,
            '目前價格': price,
            '遊戲連結': link
        })
        rank += 1  # 排名遞增
    time.sleep(3)  # 每抓一頁就停3秒

# 儲存為 CSV
df_G = pd.DataFrame(data)
df_G.to_csv('steam_global_top100.csv', index=False, encoding='utf-8-sig')

## Steam 臺灣熱門暢銷遊戲 Top 100

先確認是否可以抓取資料， 200 為正確， 403 為禁止存取， 404 為網頁不存在， 503 為服務暫時無法使用。

In [49]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

popularTW_url = 'https://store.steampowered.com/search/?cc=TW&ndl=1'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36 Edg/135.0.0.0',
       'Accept-Language': 'zh-TW,zh;q=0.9',
       'Cookie': 'steamCountry=TW; Steam_Language=tchinese'}
response = requests.get(popular_url,headers=headers)
response.status_code

200

藉由 Beautifulshop 抓取遊戲名稱、圖片網址、評價、價格、折扣幅度、遊戲連結，最後利用 pandas 製作表格存於 csv 檔中。

In [50]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# 用來儲存所有遊戲資料
data = []
# 初始排名
rank = 1

# Steam 一頁50筆，抓取前2頁共100筆
for page in range(0, 50, 25):
    popularTW_top100_url = f'https://store.steampowered.com/search/?cc=TW&ndl=1&start={page}'
    r = requests.get(popularTW_top100_url, headers=headers)
    soup = BeautifulSoup(r.text, 'lxml')
    games = soup.find_all('a', class_='search_result_row')
    for game in games:
        # 頁面連結
        link = game['href']
        # 圖片
        img_tag = game.find('div', class_='col search_capsule').find('img')
        img = img_tag['src'] if img_tag else '無圖片資訊'
        # 名稱、價格、評價都在這個區塊
        name_price_div = game.find('div', class_='responsive_search_name_combined')
        # 名稱
        name_text = name_price_div.find('div', class_='col search_name ellipsis')
        name = name_text.find('span', class_='title').text.strip() if name_text else '無名稱資訊'
        # 價格
        price_div = name_price_div.find('div', class_='col search_discount_and_price responsive_secondrow')
        if price_div.find('div', class_='discount_block search_discount_block'):
          discount = price_div.find('div', class_='discount_block search_discount_block')
          rate = discount.find('div', class_='discount_pct').text.strip() # 折扣幅度
          price_dis = discount.find('div', class_='discount_prices')
          price = price_dis.find('div', class_='discount_final_price').contents[0].strip() # 折扣後價格
        elif price_div.find('div', class_='discount_block search_discount_block no_discount'):
           no_discount = price_div.find('div', class_='discount_block search_discount_block no_discount')
           rate = '無折扣'
           price_nodis = no_discount.find('div', class_='discount_prices')
           price = price_nodis.find('div', class_='discount_final_price').contents[0].strip()
        elif price_div.find('div', class_='discount_final_price free'):
           rate = '無折扣'
           price = '免費'
        else:
           rate = '無折扣'
           price = '無價格資訊'
        # 評價
        review_div = name_price_div.find('div', class_='col search_reviewscore responsive_secondrow')
        if review_div and review_div.span and 'data-tooltip-html' in review_div.span.attrs:
            review_html = review_div.span['data-tooltip-html']
            review_rate = review_html.split('<br>')[0]
            review_tot = review_html.split('<br>')[-1]
        else:
          review_rate = '尚無評價'
          review_tot = '尚無評價'
        # 輸入資料到表格
        data.append({
            '排名': rank,
            '遊戲名稱': name,
            '圖片網址': img,
            '評價比例': review_rate,
            '總體評價': review_tot,
            '折扣幅度': rate,
            '目前價格': price,
            '遊戲連結': link
        })
        rank += 1  # 排名遞增
    time.sleep(3)  # 每抓一頁就停3秒

# 儲存為 CSV
df_TW = pd.DataFrame(data)
df_TW.to_csv('steam_TW_top100.csv', index=False, encoding='utf-8-sig')

## Steam 當前玩家最多的遊戲 Top 100

先確認是否可以抓取資料， 200 為正確， 403 為禁止存取， 404 為網頁不存在， 503 為服務暫時無法使用。

In [62]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

player_url = 'https://steamcharts.com/top'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36 Edg/135.0.0.0',
       'Accept-Language': 'zh-TW,zh;q=0.9',
       'Cookie': 'steamCountry=TW; Steam_Language=tchinese'}
response = requests.get(player_url,headers=headers)
response.status_code

200

藉由 Beautifulshop 抓取遊戲名稱、現在遊玩人數、最多遊玩人數，最後利用 pandas 製作表格存於 csv 檔中。

In [67]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# 用來儲存所有遊戲資料
data = []
# 初始排名
rank = 1

# SteamCharts 一頁25筆，抓取前4頁共100筆
for page in range(1, 5, 1):
    player_top100_url = f'https://steamcharts.com/top/p.{page}'
    r = requests.get(player_top100_url, headers=headers)
    soup = BeautifulSoup(r.text, 'lxml')
    games = soup.find_all('tr', class_='odd')
    for game in games:
        # 遊戲名稱
        name = game.find('td', class_='game-name left').find('a').text.strip()
        # 現在遊玩人數
        current_player = game.find('td', class_='num').text.strip()
        # 最多遊玩人數
        most_player = game.find('td', class_='num period-col peak-concurrent').text.strip()
        # 輸入資料到表格
        data.append({
            '排名': rank,
            '遊戲名稱': name,
            '現在遊玩人數': current_player,
            '最多遊玩人數': most_player,
        })
        rank += 1  # 排名遞增
    time.sleep(3)  # 每抓一頁就停3秒

# 儲存為 CSV
df_player = pd.DataFrame(data)
df_player.to_csv('steam_player_top100.csv', index=False, encoding='utf-8-sig')

## 結果


* 新遊戲迅速躋身排行榜，顯示出新遊戲在市場中獲得大量關注，並且通常會在發售初期快速進入銷售前列。


* 無論是全球還是台灣排行榜，多人連線遊戲佔排行榜前列，顯示出競技性與社交屬性遊戲在市場中的主導地位。


* 折扣幅度在促銷期會直接影響遊戲的銷量，許多玩家會在遊戲折扣幅度達到一定比例時進行購買，這不僅提高了遊戲的銷售量，也使得遊戲在市場上擁有更長的生命週期。

## 問題與挑戰

- 技術挑戰  
 * 需透過 AJAX 取得資料。
 * Steam 頁面結構繁複，包含多層 JavaScript 載入機制。
 * 評價、價格等欄位格式不一，不易抓取。
 * 為取得中文與新台幣資料，需模擬台灣地區 IP、Cookie 與語系參數。
- 資料限制  
 * 第三方平台資料非即時更新，可能造成與實際價格出現落差。
 * 部分新遊戲尚未累積玩家統計數據。

## 結論


本次專題成功實作多網站、多頁面、多格式的資料蒐集與整合，包含 Steam 全球與台灣區排行榜、Steam Charts 玩家人數資料及第三方平台價格資訊。整體成果可提供：

* 遊戲市場趨勢與地域偏好比較
* 定價策略與促銷時間點分析
* 消費者對熱門遊戲評價與購買行為初步洞察

未來可進一步整合更多平台如 Epic、Xbox、PlayStation Store 等，擴展成跨平台資料庫，建立更完整的遊戲行銷資料科學應用模型。

## 參考文獻

*  [Steam](https://store.steampowered.com/)
*  [IsThereAnyDeal](https://isthereanydeal.com/)
*  [Blog](https://blog.csdn.net/DDDHL_/article/details/111768725)